In [1]:
import fasttext

In [2]:
model = fasttext.load_model('/Users/lakshaywadhwa/Downloads/cc.hi.300.bin')

In [3]:
import nltk
import pandas as pd
#from sentence_transformers import SentenceTransformer
from nltk.cluster import KMeansClusterer
import numpy as np

In [4]:
import nltk

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lakshaywadhwa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
def max_only(lst):
    filtered_lst=[x for x in lst if len(x)>70]
    return filtered_lst

In [7]:
article='''
बुंदेलखंड एक्सप्रेसवे (Bundelkhand Expressway) का बताकर एक वीडियो सोशल मीडिया पर वायरल हो रहा है. वीडियो में एक टूटा हुआ पुल देखा जा सकता है. वीडियो शेयर कर बीजेपी (BJP) और पीएम नरेंद्र मोदी (Narendra Modi) पर कटाक्ष किया जा रहा है. हाल में बुंदेलखंड एक्सप्रेसवे के उद्घाटन के बाद इसके क्षतिग्रस्त होने की खबरें आई थीं. ऐसे में ये दावा वायरल हो रहा है. हालांकि, पड़ताल में हमने पाया कि वायरल वीडियो बुंदेलखंड एक्सप्रेसवे का नहीं, बल्कि मध्य प्रदेश में कलियासोत नदी पर बने पुल का है. घटना रायसेन जिले के मंडीदीप इलाके में नेशनल हाईवे 46 की है. वायरल वीडियो के की-फ्रेम्स को गूगल पर रिवर्स सर्च करने से हमें 25 जुलाई 2022 की Aaj Tak पर पब्लिश एक रिपोर्ट मिली, जिसमें इस वीडियो के स्क्रीनशॉट इस्तेमाल किए गए थे. स्टोरी के मुताबिक, मध्य प्रदेश के मंडीदीप इलाके में कलियासोत नदी में बना पुल और रोड क्षतिग्रस्त हो गए. घटना से संबंधित India Today की भी एक रिपोर्ट हमें मिली, यहां भी इस वीडियो को मध्य प्रदेश के रायसेन जिले के मंडीदीप इलाके का बताया गया है. रिपोर्ट के मुताबिक, NH46 में कलियासोत नदी पर बने इस पुल की लागत 500 करोड़ से ज्यादा बताई गई है. इस घटना का वीडियो कांग्रेस नेता कमलनाथ ने भी ट्वीट किया है. उनके कैप्शन के मुताबिक वीडियो मध्यप्रदेश में भोपाल-होशंगाबाद को जोड़ने वाले नेशनल हाईवे का है और मंडीदीप इलाके का है. हमें वायरल वीडियो का लंबा वर्जन India Today के ऑफिशियल यूट्यूब हैंडल पर मिला, जिसे 25 जुलाई को अपलोड किया गया था.
'''

In [13]:
def get_sentence_embeddings(sentence):
    embedding = model[sentence]
    return embedding

In [18]:
from scipy.spatial import distance_matrix
def distance_from_centroid(row):
    #type of emb and centroid is different, hence using tolist below
    return distance_matrix([row['embeddings']], [row['centroid'].tolist()])[0][0]
# data['distance_from_centroid'] = data.apply(distance_from_centroid, axis=1)

In [21]:
def clustering(article):
    NUM_CLUSTERS=2
    iterations=25
    sentences=nltk.sent_tokenize(article)
    # strip leading and trailing spaces
    sentences = [sentence.strip() for sentence in sentences]
    sentences=max_only(sentences)
    data = pd.DataFrame(sentences)
    data.columns=['sentence']
    data['embeddings']=data['sentence'].apply(get_sentence_embeddings)
    X = np.array(data['embeddings'].tolist())
    kclusterer = KMeansClusterer(
    NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance,
    repeats=iterations,avoid_empty_clusters=True)
    assigned_clusters = kclusterer.cluster(X, assign_clusters=True)
    data['cluster']=pd.Series(assigned_clusters, index=data.index)
    data['centroid']=data['cluster'].apply(lambda x: kclusterer.means()[x])
    data['distance_from_centroid'] = data.apply(distance_from_centroid, axis=1)
    summary=' '.join(data.sort_values('distance_from_centroid',ascending = True).groupby('cluster').head(1).sort_index()['sentence'].tolist())
    return summary
    
    
    
    
    
    
    
    
    

In [22]:
print(clustering(article))

हालांकि, पड़ताल में हमने पाया कि वायरल वीडियो बुंदेलखंड एक्सप्रेसवे का नहीं, बल्कि मध्य प्रदेश में कलियासोत नदी पर बने पुल का है. वायरल वीडियो के की-फ्रेम्स को गूगल पर रिवर्स सर्च करने से हमें 25 जुलाई 2022 की Aaj Tak पर पब्लिश एक रिपोर्ट मिली, जिसमें इस वीडियो के स्क्रीनशॉट इस्तेमाल किए गए थे.
